In [1]:
import pandas as pd
from IPython.display import display
import numpy as np

# 12.8 Повторение merge/groupby

In [2]:
#users = pd.read_csv('C:/Users/user/Documents/GitHub/Skillfactory_Alexander_Stratonov/module_3/data/users.csv', sep = '\t', encoding = 'koi8_r')
#log = pd.read_csv('C:/Users/user/Documents/GitHub/Skillfactory_Alexander_Stratonov/module_3/data/log.csv', header = None)
users = pd.read_csv('C:/Users/крис/Documents/GitHub/Skillfactory_Alexander_Stratonov/module_3/data/users.csv', sep = '\t', encoding = 'koi8_r')
log = pd.read_csv('C:/Users/крис/Documents/GitHub/Skillfactory_Alexander_Stratonov/module_3/data/log.csv', header = None)
log.columns = ['user_id', 'time', 'bet', 'win']
users.columns = ['user_id', 'mail', 'city']
log[['bet','win']] = log[['bet','win']].fillna(0)

log = log.dropna(subset = ['time'])  
log['time'] = log['time'].apply(lambda x: x[1:])  
log['time'] = pd.to_datetime(log['time']) 

def fillna_net(row):
    if row.win < 0:
        return row.win
    else:
        return row.win - row.bet

new_net = log.apply(lambda row: fillna_net(row), axis=1)  
log['net'] = new_net
users.user_id = users.user_id.apply(lambda x: x.lower())
log = log[log.user_id != '#error']
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])
log_and_users = pd.merge(users, log, on='user_id')
log_and_users.head()

,user_id,mail,city,time,bet,win,net
0,user_943,Accumanst@gmail.com,Ижевск,2019-01-06 21:06:57,0.0,0.0,0.0
1,user_943,Accumanst@gmail.com,Ижевск,2019-01-10 12:16:22,100.0,0.0,-100.0
2,user_943,Accumanst@gmail.com,Ижевск,2019-01-27 01:18:47,100.0,0.0,-100.0
3,user_943,Accumanst@gmail.com,Ижевск,2019-02-10 11:09:24,100.0,5456.0,5356.0
4,user_943,Accumanst@gmail.com,Ижевск,2019-02-12 15:09:38,300.0,5245.0,4945.0


In [3]:
log_and_users[log_and_users.win > 0].groupby('user_id').win.median().median()

5951.75

# 12.9 Анализ по пользователям

In [4]:
log_and_users.groupby('user_id').net.sum().median()

1986.0

In [5]:
log_and_users[log_and_users.bet==0].groupby('user_id').bet.value_counts().sum() / len(log_and_users.groupby('user_id'))

5.05

In [12]:
log_and_users[log_and_users.bet==0].groupby('user_id').time.min().head()

user_id
user_1000   2019-01-21 19:31:34
user_900    2019-01-01 04:37:25
user_901    2019-01-07 08:45:59
user_902    2019-01-28 03:12:06
user_903    2019-01-01 16:31:16
                    ...        
user_995    2019-01-07 09:56:20
user_996    2019-03-03 11:43:38
user_997    2019-01-05 20:55:29
user_998    2019-01-03 02:48:25
user_999    2019-01-07 10:07:52
Name: time, Length: 100, dtype: datetime64[ns]

In [13]:
log_and_users[log_and_users.bet>0].groupby('user_id').time.min().head()

user_id
user_1000   2019-01-28 20:57:37
user_900    2019-02-08 10:36:47
user_901    2019-03-14 13:51:20
user_902    2019-03-09 16:52:11
user_903    2019-02-08 11:06:00
                    ...        
user_995    2019-02-26 15:29:25
user_996    2019-01-10 07:32:17
user_997    2019-03-05 11:05:39
user_998    2019-01-25 08:57:20
user_999    2019-03-15 22:50:47
Name: time, Length: 100, dtype: datetime64[ns]

In [16]:
a=log_and_users[log_and_users.bet>0].groupby('user_id').time.min() - log_and_users[log_and_users.bet==0].groupby('user_id').time.min()
a.mean()

Timedelta('46 days 06:54:48')

# 12.10 Анализ по городам

In [7]:
log_and_users.groupby('city').win.sum().sort_values(ascending=False).head(2)

city
Москва     11959741.0
Воронеж      184338.0
Name: win, dtype: float64

In [8]:
log_and_users[log_and_users.bet > 0].groupby('city').bet.mean().max()/log_and_users[log_and_users.bet > 0].groupby('city').bet.mean().min()

127.81602335164835

In [9]:
sample2 = log_and_users.groupby('city').user_id.count()
sample2

city
Арзангелтск         96
Воронеж             88
Екатеринбург        49
Ижевск              61
Казань              66
Краснодар           86
Красноярск          56
Москва              61
Пермь               55
Санкт-Петербург    115
Ставрополь          36
Тюмень              32
Хабаровск           60
Ярославль           89
Name: user_id, dtype: int64

### НЕ ПРИНИМАЕТ КОД
users = pd.read_csv('users.csv', sep = '\t', encoding = 'koi8_r')
log = pd.read_csv('log.csv', header = None)
log.columns = ['user_id', 'time', 'bet', 'win']
users.columns = ['user_id', 'mail', 'city']
log = log[log.user_id != '#error']
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])
log_and_users = pd.merge(users, log, on='user_id')
sample2 = log_and_users.groupby('city').user_id.count()